In [2]:
!pip install snscrape

SNscrape

In [3]:
stock = input('What stock are you interested in?: ')

What stock are you interested in?: PFE


In [4]:
!pip install yfinance
import yfinance as yf

data = yf.download(  # or pdr.get_data_yahoo(...
        # tickers list or string as well
        tickers = stock,

        # use "period" instead of start/end
        # valid periods: 1d,5d,1mo,3mo,6mo,1y,2y,5y,10y,ytd,max
        # (optional, default is '1mo')
        period = "60d",

        # fetch data by interval (including intraday if period < 60 days)
        # valid intervals: 1m,2m,5m,15m,30m,60m,90m,1h,1d,5d,1wk,1mo,3mo
        # (optional, default is '1d')
        interval = "30m",

        # group by ticker (to access via data['SPY'])
        # (optional, default is 'column')
        group_by = 'ticker',

        # adjust all OHLC automatically
        # (optional, default is False)
        auto_adjust = True,

        # download pre/post regular market hours data
        # (optional, default is False)
        prepost = False,

        # use threads for mass downloading? (True/False/Integer)
        # (optional, default is True)
        threads = True,

        # proxy URL scheme use use when downloading?
        # (optional, default is None)
        proxy = None
    )

data

     |████████████████████████████████| 5.5MB 9.2MB/s 
  Created wheel for yfinance: filename=yfinance-0.1.59-py2.py3-none-any.whl size=23442 sha256=c9cdbf3d069ce15f7fb86612bf69058ed1f504dd126af6a1fd0ae3b0f4f06238
  Stored in directory: /root/.cache/pip/wheels/f8/2a/0f/4b5a86e1d52e451757eb6bc17fd899629f0925c777741b6d04
Successfully built yfinance
  Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6
[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Volume
Datetime,,,,,
2020-12-30 09:30:00-05:00,37.174999,37.217999,36.869999,37.099998,2945949
2020-12-30 10:00:00-05:00,37.099998,37.240002,37.009998,37.009998,1901644
2020-12-30 10:30:00-05:00,37.009998,37.139999,37.005001,37.090000,1440046
2020-12-30 11:00:00-05:00,37.080002,37.180000,37.070000,37.079899,996011
2020-12-30 11:30:00-05:00,37.075001,37.160000,37.020000,37.044998,1033359
...,...,...,...,...,...
2021-03-26 13:30:00-04:00,35.950001,36.080002,35.950001,36.064999,2182093
2021-03-26 14:00:00-04:00,36.063099,36.099998,36.009998,36.095001,1614327
2021-03-26 14:30:00-04:00,36.090000,36.130001,36.030102,36.099998,2005702


In [5]:
import snscrape.modules.twitter as sntwitter
import pandas as pd
import datetime

start = min(data.index)
end = max(data.index)

def workdays(d, end, excluded=(6, 7)):
    days = []
    while d.date() <= end.date():
        if d.isoweekday() not in excluded:
            days.append(d)
        d += datetime.timedelta(days=1)
    return days

total_days = workdays(datetime.datetime(start.year, start.month, start.day), datetime.datetime(end.year, end.month, end.day))
listofdates = []
for i in range (len(total_days)):
    date_time = total_days[i] + datetime.timedelta(minutes=30)
    date_time = date_time + datetime.timedelta(hours=15)
    end_time = date_time.strftime("%Y-%m-%d")  
    listofdates.append(end_time)

from datetime import datetime, timedelta
last = datetime.strptime(listofdates[-1] , '%Y-%m-%d')
last = last + timedelta(days=1)
last = last.strftime("%Y-%m-%d") 
listofdates.append(last)

# listofdates
tweets_df1 = pd.DataFrame()

for i in range(len(listofdates)):
    tweets_list = []
    if i == len(listofdates)-1:
        for i,tweet in enumerate(sntwitter.TwitterSearchScraper(str(stock) + ' lang:en since:' + str(listofdates[i]) + ' until:' + str(listofdates[i])).get_items()):
            tweets_list.append([tweet.date, tweet.content])

    else:
        for i,tweet in enumerate(sntwitter.TwitterSearchScraper(str(stock) + ' lang:en since:' + str(listofdates[i]) + ' until:' + str(listofdates[i+1])).get_items()):
            tweets_list.append([tweet.date, tweet.content])

    tweets_df = pd.DataFrame(tweets_list, columns=['Datetime', 'Text'])
    tweets_df1 = tweets_df1.append(tweets_df[::-1]) 
  

In [6]:
tweets_df1

,Datetime,Text
375,2020-12-30 00:02:19+00:00,Sellers Weaken the Stock Market’s Hold On Over...
374,2020-12-30 00:03:41+00:00,$DECN #Covid19 Testing Is Pretty Much Old News...
373,2020-12-30 00:10:08+00:00,@PersimmonTI $tril \nToo bad Jan's reply wasn...
372,2020-12-30 00:13:01+00:00,#Epilepsy Market Carries Exceptional Growth $9...
371,2020-12-30 00:22:01+00:00,"@megtirrell Makes sense, but simultaneously 1a..."
...,...,...
4,2021-03-26 23:30:00+00:00,CHECKING IN ON .@JIMCRAMER PREDICTIONS\n\n#STO...
3,2021-03-26 23:45:22+00:00,The Value Stock Rotation Isn’t Over—Not by a L...
2,2021-03-26 23:47:41+00:00,【Get rich in 100 days】\nDay 18☺️\n\nDOW⤴️NAS⤴️...
1,2021-03-26 23:48:27+00:00,$nvax $mrna $pfe another article on the subjec...


In [7]:
import pandas as pd
import numpy as np

In [8]:
!pip install emoji

     |████████████████████████████████| 133kB 6.3MB/s 


In [9]:
import re
import emoji
tweets_df1['Text'] = tweets_df1['Text'].astype(str)

def cleaner(tweet):
    tweet = re.sub(r"(^|\W)\d+", "", tweet)# remove digits
    tweet = tweet.lower()
    if tweet[:2] == 'rt':
        tweet = re.sub(r'^.*?:', ':', tweet)
    tweet = re.sub("@[A-Za-z0-9]+","",tweet) #Remove @ sign
    tweet = re.sub(r"(?:\@|http?\://|https?\://|www)\S+", "", tweet) #Remove http links
    tweet = " ".join(tweet.split())
    tweet = ''.join(c for c in tweet if c not in emoji.UNICODE_EMOJI) #Remove Emojis
    tweet = tweet.replace("#", "").replace("_", " ")
    tweet = tweet.replace("RT ", "")
    tweet = re.sub(r'[^\w\s]','',tweet) #remove punctuation
    return tweet
tweets_df1['processed_tweet'] = tweets_df1['Text'].map(lambda x: cleaner(x))

In [10]:
tweets_df1

,Datetime,Text,processed_tweet
375,2020-12-30 00:02:19+00:00,Sellers Weaken the Stock Market’s Hold On Over...,sellers weaken the stock markets hold on overb...
374,2020-12-30 00:03:41+00:00,$DECN #Covid19 Testing Is Pretty Much Old News...,decn covid19 testing is pretty much old news t...
373,2020-12-30 00:10:08+00:00,@PersimmonTI $tril \nToo bad Jan's reply wasn...,tril too bad jans reply wasnt ill call pfe and...
372,2020-12-30 00:13:01+00:00,#Epilepsy Market Carries Exceptional Growth $9...,epilepsy market carries exceptional growth b b...
371,2020-12-30 00:22:01+00:00,"@megtirrell Makes sense, but simultaneously 1a...",makes sense but simultaneouslya should be vacc...
...,...,...,...
4,2021-03-26 23:30:00+00:00,CHECKING IN ON .@JIMCRAMER PREDICTIONS\n\n#STO...,checking in on predictions stockstobuy cost ...
3,2021-03-26 23:45:22+00:00,The Value Stock Rotation Isn’t Over—Not by a L...,the value stock rotation isnt overnot by a lon...
2,2021-03-26 23:47:41+00:00,【Get rich in 100 days】\nDay 18☺️\n\nDOW⤴️NAS⤴️...,get rich in days day downasmy pf i never cut m...
1,2021-03-26 23:48:27+00:00,$nvax $mrna $pfe another article on the subjec...,nvax mrna pfe another article on the subject t...


In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

/usr/local/lib/python3.7/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [12]:
prices = data.reset_index()
prices.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 780 entries, 0 to 779
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype                           
---  ------    --------------  -----                           
 0   Datetime  780 non-null    datetime64[ns, America/New_York]
 1   Open      780 non-null    float64                         
 2   High      780 non-null    float64                         
 3   Low       780 non-null    float64                         
 4   Close     780 non-null    float64                         
 5   Volume    780 non-null    int64                           
dtypes: datetime64[ns, America/New_York](1), float64(4), int64(1)
memory usage: 36.7 KB


In [13]:
prices['Datetime'] = prices['Datetime'].astype(str)

In [14]:
#remove -5:00
prices['Datetime'] = prices['Datetime'].str.replace('-05:00',"")
prices['Datetime'] = prices['Datetime'].str.replace('-04:00',"")

In [15]:
prices = prices[(prices['Datetime']>= min(prices['Datetime'])) & (prices['Datetime']<=max(prices['Datetime']))]
prices

,Datetime,Open,High,Low,Close,Volume
0,2020-12-30 09:30:00,37.174999,37.217999,36.869999,37.099998,2945949
1,2020-12-30 10:00:00,37.099998,37.240002,37.009998,37.009998,1901644
2,2020-12-30 10:30:00,37.009998,37.139999,37.005001,37.090000,1440046
3,2020-12-30 11:00:00,37.080002,37.180000,37.070000,37.079899,996011
4,2020-12-30 11:30:00,37.075001,37.160000,37.020000,37.044998,1033359
...,...,...,...,...,...,...
775,2021-03-26 13:30:00,35.950001,36.080002,35.950001,36.064999,2182093
776,2021-03-26 14:00:00,36.063099,36.099998,36.009998,36.095001,1614327
777,2021-03-26 14:30:00,36.090000,36.130001,36.030102,36.099998,2005702
778,2021-03-26 15:00:00,36.095001,36.125000,36.020000,36.040001,1353892


In [16]:
sentiment = tweets_df1

In [17]:
sentiment = sentiment.rename(columns={'Datetime': 'created_at'})

In [18]:
#remove +00:00
sentiment['created_at'] = sentiment['created_at'].astype(str)
sentiment['created_at'] = sentiment['created_at'].str.slice(0,19)
sentiment.head()

,created_at,Text,processed_tweet
375,2020-12-30 00:02:19,Sellers Weaken the Stock Market’s Hold On Over...,sellers weaken the stock markets hold on overb...
374,2020-12-30 00:03:41,$DECN #Covid19 Testing Is Pretty Much Old News...,decn covid19 testing is pretty much old news t...
373,2020-12-30 00:10:08,@PersimmonTI $tril \nToo bad Jan's reply wasn...,tril too bad jans reply wasnt ill call pfe and...
372,2020-12-30 00:13:01,#Epilepsy Market Carries Exceptional Growth $9...,epilepsy market carries exceptional growth b b...
371,2020-12-30 00:22:01,"@megtirrell Makes sense, but simultaneously 1a...",makes sense but simultaneouslya should be vacc...


In [19]:
# convert datetime and created at to same datetime object
prices['Datetime'] = pd.to_datetime(prices['Datetime'])
sentiment['created_at'] = pd.to_datetime(sentiment['created_at'])

In [20]:
# only close column is needed
prices = prices.drop(columns=['Open','High','Low','Volume'])

In [21]:
sentiment = sentiment.rename(columns={'created_at': 'Datetime'})
sentiment

,Datetime,Text,processed_tweet
375,2020-12-30 00:02:19,Sellers Weaken the Stock Market’s Hold On Over...,sellers weaken the stock markets hold on overb...
374,2020-12-30 00:03:41,$DECN #Covid19 Testing Is Pretty Much Old News...,decn covid19 testing is pretty much old news t...
373,2020-12-30 00:10:08,@PersimmonTI $tril \nToo bad Jan's reply wasn...,tril too bad jans reply wasnt ill call pfe and...
372,2020-12-30 00:13:01,#Epilepsy Market Carries Exceptional Growth $9...,epilepsy market carries exceptional growth b b...
371,2020-12-30 00:22:01,"@megtirrell Makes sense, but simultaneously 1a...",makes sense but simultaneouslya should be vacc...
...,...,...,...
4,2021-03-26 23:30:00,CHECKING IN ON .@JIMCRAMER PREDICTIONS\n\n#STO...,checking in on predictions stockstobuy cost ...
3,2021-03-26 23:45:22,The Value Stock Rotation Isn’t Over—Not by a L...,the value stock rotation isnt overnot by a lon...
2,2021-03-26 23:47:41,【Get rich in 100 days】\nDay 18☺️\n\nDOW⤴️NAS⤴️...,get rich in days day downasmy pf i never cut m...
1,2021-03-26 23:48:27,$nvax $mrna $pfe another article on the subjec...,nvax mrna pfe another article on the subject t...


In [22]:
# drop na of sentiment
sentiment = sentiment.dropna()

In [23]:
df_merge = pd.merge_asof(sentiment,prices,left_on='Datetime',right_on="Datetime",direction='backward')
df_merge

,Datetime,Text,processed_tweet,Close
0,2020-12-30 00:02:19,Sellers Weaken the Stock Market’s Hold On Over...,sellers weaken the stock markets hold on overb...,NaN
1,2020-12-30 00:03:41,$DECN #Covid19 Testing Is Pretty Much Old News...,decn covid19 testing is pretty much old news t...,NaN
2,2020-12-30 00:10:08,@PersimmonTI $tril \nToo bad Jan's reply wasn...,tril too bad jans reply wasnt ill call pfe and...,NaN
3,2020-12-30 00:13:01,#Epilepsy Market Carries Exceptional Growth $9...,epilepsy market carries exceptional growth b b...,NaN
4,2020-12-30 00:22:01,"@megtirrell Makes sense, but simultaneously 1a...",makes sense but simultaneouslya should be vacc...,NaN
...,...,...,...,...
23765,2021-03-26 23:30:00,CHECKING IN ON .@JIMCRAMER PREDICTIONS\n\n#STO...,checking in on predictions stockstobuy cost ...,36.25
23766,2021-03-26 23:45:22,The Value Stock Rotation Isn’t Over—Not by a L...,the value stock rotation isnt overnot by a lon...,36.25
23767,2021-03-26 23:47:41,【Get rich in 100 days】\nDay 18☺️\n\nDOW⤴️NAS⤴️...,get rich in days day downasmy pf i never cut m...,36.25
23768,2021-03-26 23:48:27,$nvax $mrna $pfe another article on the subjec...,nvax mrna pfe another article on the subject t...,36.25


In [24]:
df_merge1 = df_merge[df_merge.Close.notnull()]
df_merge1

,Datetime,Text,processed_tweet,Close
99,2020-12-30 09:33:47,@DalhiMackle @RNAiAnalyst 1) $NVAX scale up is...,nvax scale up issue which caused a month dela...,37.099998
100,2020-12-30 09:39:42,@sciencescanner I'm not sure this is advice a ...,im not sure this is advice a doctor or pharmac...,37.099998
101,2020-12-30 09:41:01,The kind of news you love to start the day. Be...,the kind of news you love to start the day bet...,37.099998
102,2020-12-30 10:04:22,@iangtobin @sciencescanner Hardly anyone will ...,hardly anyone will be better off waiting espe...,37.009998
103,2020-12-30 10:12:40,"@alabamawins17 It’s hard to tell, MRNA has alr...",its hard to tell mrna has already had a huge c...,37.009998
...,...,...,...,...
23765,2021-03-26 23:30:00,CHECKING IN ON .@JIMCRAMER PREDICTIONS\n\n#STO...,checking in on predictions stockstobuy cost ...,36.250000
23766,2021-03-26 23:45:22,The Value Stock Rotation Isn’t Over—Not by a L...,the value stock rotation isnt overnot by a lon...,36.250000
23767,2021-03-26 23:47:41,【Get rich in 100 days】\nDay 18☺️\n\nDOW⤴️NAS⤴️...,get rich in days day downasmy pf i never cut m...,36.250000
23768,2021-03-26 23:48:27,$nvax $mrna $pfe another article on the subjec...,nvax mrna pfe another article on the subject t...,36.250000


In [25]:
import datetime

df_final = pd.DataFrame()

start = min(data.index)
end = max(data.index)

def workdays(d, end, excluded=(6, 7)):
    days = []
    while d.date() <= end.date():
        if d.isoweekday() not in excluded:
            days.append(d)
        d += datetime.timedelta(days=1)
    return days

total_days = workdays(datetime.datetime(start.year, start.month, start.day), datetime.datetime(end.year, end.month, end.day))
listofdates = []
for i in range (len(total_days)):
    date_time = total_days[i] + datetime.timedelta(minutes=30)
    date_time = date_time + datetime.timedelta(hours=15)
    end_time = date_time.strftime("%Y-%m-%d %H:%M:%S")

    date_time2 = total_days[i] + datetime.timedelta(hours=9)
    start_time = date_time2.strftime("%Y-%m-%d %H:%M:%S")

    listofdates.append([end_time, start_time])

# print(len(total_days))

for i in range(len(total_days)):
    df_final = df_final.append(df_merge1[(df_merge1['Datetime']<=listofdates[i][0]) & (df_merge1['Datetime']>=listofdates[i][1])])

In [26]:
df_final

,Datetime,Text,processed_tweet,Close
99,2020-12-30 09:33:47,@DalhiMackle @RNAiAnalyst 1) $NVAX scale up is...,nvax scale up issue which caused a month dela...,37.099998
100,2020-12-30 09:39:42,@sciencescanner I'm not sure this is advice a ...,im not sure this is advice a doctor or pharmac...,37.099998
101,2020-12-30 09:41:01,The kind of news you love to start the day. Be...,the kind of news you love to start the day bet...,37.099998
102,2020-12-30 10:04:22,@iangtobin @sciencescanner Hardly anyone will ...,hardly anyone will be better off waiting espe...,37.009998
103,2020-12-30 10:12:40,"@alabamawins17 It’s hard to tell, MRNA has alr...",its hard to tell mrna has already had a huge c...,37.009998
...,...,...,...,...
23624,2021-03-26 15:00:07,S&amp;P 500 Index Update 📈\n\nCharts are relea...,sampp index update charts are released earlie...,36.040001
23625,2021-03-26 15:04:51,$PFE Has A Poor #Technical Analysis Score (TA ...,pfe has a poor technical analysis score ta sco...,36.040001
23626,2021-03-26 15:23:12,$BNTX: BioNTech and Pfizer (PFE) receive EMA a...,bntx biontech and pfizer pfe receive ema appro...,36.040001
23627,2021-03-26 15:24:13,"Today Top Flow in S&amp;P 500 #SP500, Buy Flow...",today top flow in sampp sp500 buy flow amp sel...,36.040001


In [27]:
df_final = df_final.reset_index(drop=True)
df_final

,Datetime,Text,processed_tweet,Close
0,2020-12-30 09:33:47,@DalhiMackle @RNAiAnalyst 1) $NVAX scale up is...,nvax scale up issue which caused a month dela...,37.099998
1,2020-12-30 09:39:42,@sciencescanner I'm not sure this is advice a ...,im not sure this is advice a doctor or pharmac...,37.099998
2,2020-12-30 09:41:01,The kind of news you love to start the day. Be...,the kind of news you love to start the day bet...,37.099998
3,2020-12-30 10:04:22,@iangtobin @sciencescanner Hardly anyone will ...,hardly anyone will be better off waiting espe...,37.009998
4,2020-12-30 10:12:40,"@alabamawins17 It’s hard to tell, MRNA has alr...",its hard to tell mrna has already had a huge c...,37.009998
...,...,...,...,...
6438,2021-03-26 15:00:07,S&amp;P 500 Index Update 📈\n\nCharts are relea...,sampp index update charts are released earlie...,36.040001
6439,2021-03-26 15:04:51,$PFE Has A Poor #Technical Analysis Score (TA ...,pfe has a poor technical analysis score ta sco...,36.040001
6440,2021-03-26 15:23:12,$BNTX: BioNTech and Pfizer (PFE) receive EMA a...,bntx biontech and pfizer pfe receive ema appro...,36.040001
6441,2021-03-26 15:24:13,"Today Top Flow in S&amp;P 500 #SP500, Buy Flow...",today top flow in sampp sp500 buy flow amp sel...,36.040001


In [28]:
df_final["Comp"] = ''
df_final["Negative"] = ''
df_final["Neutral"] = ''
df_final["Positive"] = ''
df_final

,Datetime,Text,processed_tweet,Close,Comp,Negative,Neutral,Positive
0,2020-12-30 09:33:47,@DalhiMackle @RNAiAnalyst 1) $NVAX scale up is...,nvax scale up issue which caused a month dela...,37.099998,,,,
1,2020-12-30 09:39:42,@sciencescanner I'm not sure this is advice a ...,im not sure this is advice a doctor or pharmac...,37.099998,,,,
2,2020-12-30 09:41:01,The kind of news you love to start the day. Be...,the kind of news you love to start the day bet...,37.099998,,,,
3,2020-12-30 10:04:22,@iangtobin @sciencescanner Hardly anyone will ...,hardly anyone will be better off waiting espe...,37.009998,,,,
4,2020-12-30 10:12:40,"@alabamawins17 It’s hard to tell, MRNA has alr...",its hard to tell mrna has already had a huge c...,37.009998,,,,
...,...,...,...,...,...,...,...,...
6438,2021-03-26 15:00:07,S&amp;P 500 Index Update 📈\n\nCharts are relea...,sampp index update charts are released earlie...,36.040001,,,,
6439,2021-03-26 15:04:51,$PFE Has A Poor #Technical Analysis Score (TA ...,pfe has a poor technical analysis score ta sco...,36.040001,,,,
6440,2021-03-26 15:23:12,$BNTX: BioNTech and Pfizer (PFE) receive EMA a...,bntx biontech and pfizer pfe receive ema appro...,36.040001,,,,
6441,2021-03-26 15:24:13,"Today Top Flow in S&amp;P 500 #SP500, Buy Flow...",today top flow in sampp sp500 buy flow amp sel...,36.040001,,,,


In [29]:
df_final['processed_tweet'] = df_final['processed_tweet'].astype(str)
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6443 entries, 0 to 6442
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Datetime         6443 non-null   datetime64[ns]
 1   Text             6443 non-null   object        
 2   processed_tweet  6443 non-null   object        
 3   Close            6443 non-null   float64       
 4   Comp             6443 non-null   object        
 5   Negative         6443 non-null   object        
 6   Neutral          6443 non-null   object        
 7   Positive         6443 non-null   object        
dtypes: datetime64[ns](1), float64(1), object(6)
memory usage: 402.8+ KB


In [30]:
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer

analyzer = SentimentIntensityAnalyzer()
df_final['Comp'] = [analyzer.polarity_scores(x)['compound'] for x in df_final['processed_tweet']]
df_final['Negative'] = [analyzer.polarity_scores(x)['neg'] for x in df_final['processed_tweet']]
df_final['Neutral'] = [analyzer.polarity_scores(x)['neu'] for x in df_final['processed_tweet']]
df_final['Positive'] = [analyzer.polarity_scores(x)['pos'] for x in df_final['processed_tweet']]
df_final

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


,Datetime,Text,processed_tweet,Close,Comp,Negative,Neutral,Positive
0,2020-12-30 09:33:47,@DalhiMackle @RNAiAnalyst 1) $NVAX scale up is...,nvax scale up issue which caused a month dela...,37.099998,0.5994,0.080,0.765,0.155
1,2020-12-30 09:39:42,@sciencescanner I'm not sure this is advice a ...,im not sure this is advice a doctor or pharmac...,37.099998,0.6639,0.042,0.820,0.138
2,2020-12-30 09:41:01,The kind of news you love to start the day. Be...,the kind of news you love to start the day bet...,37.099998,0.7430,0.000,0.867,0.133
3,2020-12-30 10:04:22,@iangtobin @sciencescanner Hardly anyone will ...,hardly anyone will be better off waiting espe...,37.009998,0.4019,0.047,0.849,0.104
4,2020-12-30 10:12:40,"@alabamawins17 It’s hard to tell, MRNA has alr...",its hard to tell mrna has already had a huge c...,37.009998,0.2263,0.059,0.844,0.097
...,...,...,...,...,...,...,...,...
6438,2021-03-26 15:00:07,S&amp;P 500 Index Update 📈\n\nCharts are relea...,sampp index update charts are released earlie...,36.040001,0.5719,0.091,0.731,0.177
6439,2021-03-26 15:04:51,$PFE Has A Poor #Technical Analysis Score (TA ...,pfe has a poor technical analysis score ta sco...,36.040001,-0.7351,0.205,0.795,0.000
6440,2021-03-26 15:23:12,$BNTX: BioNTech and Pfizer (PFE) receive EMA a...,bntx biontech and pfizer pfe receive ema appro...,36.040001,0.4767,0.000,0.807,0.193
6441,2021-03-26 15:24:13,"Today Top Flow in S&amp;P 500 #SP500, Buy Flow...",today top flow in sampp sp500 buy flow amp sel...,36.040001,0.2023,0.000,0.951,0.049


In [31]:
df_final = df_final.groupby(df_final.Datetime.dt.floor('30min')).mean()
df_final

,Close,Comp,Negative,Neutral,Positive
Datetime,,,,,
2020-12-30 09:30:00,37.099998,0.668767,0.040667,0.817333,0.142000
2020-12-30 10:00:00,37.009998,0.143571,0.032429,0.912571,0.055000
2020-12-30 10:30:00,37.090000,0.317000,0.000000,0.845667,0.154333
2020-12-30 11:00:00,37.079899,0.047514,0.019143,0.938857,0.042000
2020-12-30 11:30:00,37.044998,0.317800,0.000000,0.863333,0.136667
...,...,...,...,...,...
2021-03-26 13:00:00,35.950001,0.415055,0.020000,0.820091,0.159909
2021-03-26 13:30:00,36.064999,0.270407,0.011786,0.910429,0.077786
2021-03-26 14:00:00,36.095001,0.342650,0.028583,0.842583,0.128833


In [32]:
df_final = df_final.drop(columns=['Negative','Neutral','Positive'])
df_final

,Close,Comp
Datetime,,
2020-12-30 09:30:00,37.099998,0.668767
2020-12-30 10:00:00,37.009998,0.143571
2020-12-30 10:30:00,37.090000,0.317000
2020-12-30 11:00:00,37.079899,0.047514
2020-12-30 11:30:00,37.044998,0.317800
...,...,...
2021-03-26 13:00:00,35.950001,0.415055
2021-03-26 13:30:00,36.064999,0.270407
2021-03-26 14:00:00,36.095001,0.342650


In [33]:
df_final.to_csv('final.csv')